In [46]:
!pip3 install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
     |████████████████████████████████| 24.9 MB 23 kB/s  eta 0:00:01    |█████▎                          | 4.1 MB 2.7 MB/s eta 0:00:08
     |████████████████████████████████| 27.2 MB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 2.0 MB/s eta 0:00:01
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=b54ee409aa46e3532361a592be193015036a4df36dec02dbf191878a6ccb66d8
  Stored in directory: /home/nima/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [3]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
from sklearn.cluster import KMeans

In [4]:
def fix_image_to_show(img):
    image = img.copy()
    image = image.astype('float32')
    max_pixel = np.max(image)
    min_pixel = np.min(image)
    if max_pixel == min_pixel:
        image[:, :, :] = 127
        image = image.astype('uint8')
        return image
    m = 255 / (max_pixel - min_pixel)
    image = image * m - min_pixel * m
    image = image.astype('uint8')
    return image

def show_image(img, name=None, cmap=None):
    image = img.copy()
    image = fix_image_to_show(image)
    plt.imshow(image, vmin=0, vmax=255, cmap=cmap)
    plt.show()
    if name is not None :
        plt.imsave(name + '.jpg', image, cmap=cmap)

In [5]:
import os 
directory = 'resources/Data/Train'
files = os.listdir(directory)
images = []
class_list = []
for file in files:
    path = directory + '/' + file 
    image_names = os.listdir(path)
    for image_name in image_names: 
        image = cv2.imread(path + '/' + image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(image)
        class_list.append(file)

In [61]:
# sift = cv2.SIFT_create()
# kp1, desc1 = sift.detectAndCompute(image1, None)

# extract feature vectors from all images 

min_kp = 100000

feature_vectors = []

sift = cv2.SIFT_create()

deb = False

n = 2 # 2n + 1

for image in images: 
    kp, desc = sift.detectAndCompute(image, None)
    for i in range(min(len(kp), min_kp)):
        feature_vectors.append(desc[i])
print(len(feature_vectors))

1540388


In [62]:
kmeans = KMeans(n_clusters=100, max_iter=300)
feature_vectors = np.asarray(feature_vectors, dtype='uint8')
kmeans.fit(feature_vectors)
kmeans.labels_

array([14, 14, 72, ..., 66, 17, 43], dtype=int32)

In [63]:
kmeans.cluster_centers_

array([[19.95703009, 26.67173071, 28.45248734, ...,  8.72184987,
        10.53224605, 20.87257968],
       [13.57895416, 14.45309175, 17.90618351, ...,  7.08820685,
        10.39744664, 11.61544909],
       [13.09318477, 11.99305503, 17.48920095, ..., 14.60917347,
        12.88941464, 10.54369228],
       ...,
       [12.33400777,  7.58477053,  9.26675659, ...,  4.65712818,
         3.44580403,  7.58554132],
       [23.43721169, 20.62826756, 20.60994362, ..., 10.38307278,
        16.42414147, 16.30189646],
       [52.68442086, 13.76731207, 10.46558258, ..., 11.21642665,
        10.76124854, 18.15200165]])

In [64]:
def L1_distance(image1, image2): 
    abs_image = np.abs(image1 - image2)
    return np.sum(abs_image)

def L2_distance(image1, image2):
    dif_image = np.power(image1 - image2, 2)
    return np.sum(dif_image)



centers = kmeans.cluster_centers_


sift = cv2.SIFT_create()

histograms = []

for image in images: 
    kp, desc = sift.detectAndCompute(image, None)
    hist = np.zeros((len(centers)), dtype='uint32')
    for i in range(min(min_kp, len(kp))):
        cen = list(range(len(centers)))
        mn = min(cen, key=lambda x: L1_distance(centers[x], desc[i]))
        hist[mn] = hist[mn] + 1
        
    hist = hist.astype('float32')
    hist /= np.sum(hist)
    histograms.append(hist)


In [65]:
import os 
directory = 'resources/Data/Test'
files = os.listdir(directory)
test_images = []
test_class_list = []
for file in files:
    path = directory + '/' + file 
    image_names = os.listdir(path)
    for image_name in image_names: 
        image = cv2.imread(path + '/' + image_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        test_images.append(image)
        test_class_list.append(file)
        
        

In [66]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X = histograms.copy()
Y = class_list


from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X, Y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [67]:
def predict_class_svm(image): 
    global clf 
    kp, desc = sift.detectAndCompute(image, None)
    hist = np.zeros((len(centers)), dtype='uint32')
    for i in range(min(min_kp, len(kp))):
        
        # now we have vector 
        cen = list(range(len(centers)))
#         cen.sort(key=lambda x: L2_distance(centers[x], desc[i])
        mn = min(cen, key=lambda x: L1_distance(centers[x], desc[i]))
        hist[mn] = hist[mn] + 1
    hist = hist.astype('float32')
    hist /= np.sum(hist)
    return clf.predict(hist.reshape(1, -1))

In [68]:
correct = 0
for i in range(len(test_images)):
    image = test_images[i]
    image_class = test_class_list[i]
    predict = predict_class_svm(image)[0]
    if predict == image_class:
        correct = correct + 1
print(correct)

793


In [69]:
print(correct / len(test_images) * 100)

52.86666666666666
